<a href="https://colab.research.google.com/github/datascience-uniandes/classification_tutorial/blob/master/churn/churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification: Detecting churn probability and causes

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
Last update: October, 2023

In [ ]:
!pip show shap

In [ ]:
from joblib import dump

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import ks_2samp
from sklearn.feature_selection import chi2

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, auc

#from pandas_profiling import ProfileReport

import shap

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

### Loading the data

In [ ]:
churn_df = pd.read_csv("./data/churn_train_val.csv")
test_df = pd.read_csv("./data/churn_test_labeled.csv")

In [ ]:
churn_df.shape

In [ ]:
test_df.shape

In [ ]:
churn_df.dtypes

In [ ]:
churn_df.head()

In [ ]:
test_df.head()

### Profiling the data

In [ ]:
#profile = ProfileReport(churn_df)

In [ ]:
#profile.to_notebook_iframe()

### Analyzing relationships between features and target

In [ ]:
churn_df["churn"].value_counts(dropna=False, normalize=True)

In [ ]:
plt.figure(figsize=(15, 3))
sns.boxplot(data=churn_df, x="credit_score", y="churn", showmeans=True, orient="h")
plt.title("Credit score")
plt.show()

In [ ]:
pd.crosstab(churn_df["churn"], churn_df["country"], normalize="columns")

In [ ]:
pd.crosstab(churn_df["churn"], churn_df["gender"], normalize="columns")

In [ ]:
plt.figure(figsize=(15, 3))
sns.boxplot(data=churn_df, x="age", y="churn", showmeans=True, orient="h")
plt.title("Age")
plt.show()

In [ ]:
plt.figure(figsize=(15, 3))
sns.boxplot(data=churn_df, x="tenure", y="churn", showmeans=True, orient="h")
plt.title("Tenure")
plt.show()

In [ ]:
plt.figure(figsize=(15, 3))
sns.boxplot(data=churn_df, x="balance", y="churn", showmeans=True, orient="h")
plt.title("Balance")
plt.show()

In [ ]:
pd.crosstab(churn_df["churn"], churn_df["products_number"], normalize="columns")

In [ ]:
pd.crosstab(churn_df["churn"], churn_df["credit_card"], normalize="columns")

In [ ]:
pd.crosstab(churn_df["churn"], churn_df["active_member"], normalize="columns")

In [ ]:
plt.figure(figsize=(15, 3))
sns.boxplot(data=churn_df, x="estimated_salary", showmeans=True, y="churn", orient="h")
plt.title("Estimated salary")
plt.show()

### Quantifying the relationships

In [ ]:
numerical_features = ["credit_score", "age", "tenure", "balance", "estimated_salary"]

In [ ]:
ks_results = []
for f in numerical_features:
    ks_stat, p_value = ks_2samp(churn_df.loc[churn_df["churn"] == 1, f], churn_df.loc[churn_df["churn"] == 0, f])
    ks_results.append({
        "Feature": f,
        "KS stat": ks_stat,
        "p-value": p_value
    })
pd.DataFrame(ks_results)

- A high KS statistic (and a small p-value) indicates that the distributions of the feature are significantly different between the two classes. This suggests that the feature might be discriminative and useful for classification.
- Conversely, a low KS statistic (and a large p-value) suggests that the feature has a similar distribution across both classes and may not be as informative.

In [ ]:
categorical_features = ["country", "gender", "products_number", "credit_card", "active_member"]

In [ ]:
chi2_results = []
for f in categorical_features:
    onehot = OneHotEncoder(sparse=False)
    t = onehot.fit_transform(churn_df[[f]])
    values = onehot.categories_[0]
    chi2_stats, p_values = chi2(t, churn_df["churn"])
    
    for value, chi_stat, p_value in zip(values, chi2_stats, p_values):
        chi2_results.append({
            "Feature": f,
            "Value": value,
            "Chi2 stat": chi_stat,
            "p-value": p_value
        })
pd.DataFrame(chi2_results)

- A feature with a high χ² value and a low p-value is considered to be more relevant for the classification task because it has a significant association with the target variable. Such a feature can potentially improve the performance of a classifier.
- Conversely, a feature with a low χ² value and a high p-value might not be very informative for the classification task, as it doesn't seem to have a strong relationship with the target variable.

### Training a first set of models and selecting the best using F1

In [ ]:
features = ["age", "balance", "country", "products_number", "active_member"]

In [ ]:
pipeline = Pipeline([
    ("transformer", ColumnTransformer([
        ("categorical", OneHotEncoder(sparse=False), ["country"])
    ], remainder="passthrough")),
    ("classifier",  RandomForestClassifier(random_state=10))
]) 

In [ ]:
param_grid = {
    "classifier__n_estimators": [50, 100, 150],
    "classifier__max_depth" : [3, 4, 5, 6],
    "classifier__class_weight": ["balanced", None]
}

In [ ]:
grid = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring=["precision", "recall", "f1", "roc_auc"], refit="f1", cv=5, return_train_score=True)

In [ ]:
grid.fit(churn_df[features], churn_df["churn"])

In [ ]:
grid.best_params_

In [ ]:
best_results_df = pd.DataFrame(grid.cv_results_).iloc[grid.best_index_].reset_index().rename(columns={"index": "result", grid.best_index_: "value"})
best_results_df = best_results_df.loc[best_results_df["result"].str.contains("split")]
[best_results_df["split"], best_results_df["dataset"], best_results_df["metric"]] = zip(*best_results_df["result"].str.split("_"))
best_results_df["dataset"].replace({"test": "validation"}, inplace=True)
del best_results_df["result"]

In [ ]:
plt.figure(figsize=(8, 10))
sns.boxplot(data=best_results_df, y="value", x="metric", hue="dataset", showmeans=True)
plt.show()

In [ ]:
train_val_preds = grid.best_estimator_.predict(churn_df[features])
test_preds = grid.best_estimator_.predict(test_df[features])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

train_val_cm = confusion_matrix(churn_df["churn"], train_val_preds, labels=grid.best_estimator_.classes_, normalize="true")
train_val_disp = ConfusionMatrixDisplay(confusion_matrix=train_val_cm, display_labels=grid.best_estimator_.classes_)
train_val_disp.plot(ax=axes[0])
axes[0].set_title("Train/Val")

test_cm = confusion_matrix(test_df["churn"], test_preds, labels=grid.best_estimator_.classes_, normalize="true")
test_disp = ConfusionMatrixDisplay(confusion_matrix=test_cm, display_labels=grid.best_estimator_.classes_)
test_disp.plot(ax=axes[1])
axes[1].set_title("Test")

plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

fpr, tpr, thresholds = roc_curve(churn_df["churn"], train_val_preds)
area = auc(fpr, tpr)
train_val_disp = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=area)
train_val_disp.plot(ax=axes[0])
axes[0].set_title("Train/Val")

fpr, tpr, thresholds = roc_curve(test_df["churn"], test_preds)
area = auc(fpr, tpr)
test_disp = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=area)
test_disp.plot(ax=axes[1])
axes[1].set_title("Test")

In [ ]:
print("Precision:")
print("- Train/Val:", precision_score(churn_df["churn"], train_val_preds))
print("- Test:", precision_score(test_df["churn"], test_preds))
print("\nRecall:")
print("- Train/Val:", recall_score(churn_df["churn"], train_val_preds))
print("- Test:", recall_score(test_df["churn"], test_preds))
print("\nF1:")
print("- Train/Val:", f1_score(churn_df["churn"], train_val_preds))
print("- Test:", f1_score(test_df["churn"], test_preds))
print("\nROC AUC:")
print("- Train/Val:", roc_auc_score(churn_df["churn"], train_val_preds))
print("- Test:", roc_auc_score(test_df["churn"], test_preds))

In [ ]:
test_probs = grid.best_estimator_.predict_proba(test_df[features])[:,1]
probs_true_df = pd.DataFrame(np.append(test_probs.reshape(-1, 1), test_df["churn"].values.reshape(-1, 1), axis=1), columns=["probs", "true"])

In [ ]:
plt.figure(figsize=(15, 9))
sns.kdeplot(data=probs_true_df.loc[probs_true_df["true"] == 1], x="probs", bw_adjust=.3, label="1")
sns.kdeplot(data=probs_true_df.loc[probs_true_df["true"] == 0], x="probs", bw_adjust=.3, label="0")
plt.axvline(x=.5, color="r", linestyle="--")
plt.legend()
plt.show()

### Exporting the pipeline

In [ ]:
dump(grid.best_estimator_, "./churn-v1.0.joblib")

### Explaining predictions

In [ ]:
test_df["prob"] = test_probs

In [ ]:
test_df.sort_values(by="prob").head()

In [ ]:
test_df.sort_values(by="prob", ascending=False).head()

In [ ]:
test_df.loc[(test_df["prob"] > 0.47) & (test_df["prob"] < 0.53)].head()

In [ ]:
X_t = pd.DataFrame(
    grid.best_estimator_["transformer"].fit_transform(test_df[features]),
    columns=[f.split("__")[1] for f in grid.best_estimator_["transformer"].get_feature_names_out()]
)

In [ ]:
def model(X):
    return grid.best_estimator_["classifier"].predict_proba(X)[:,1]

explainer = shap.Explainer(model, X_t)
shap_values = explainer(X_t)

**Explaining individual predictions:**

In [ ]:
shap.plots.waterfall(shap_values[837])

In [ ]:
shap.plots.waterfall(shap_values[148])

In [ ]:
shap.plots.waterfall(shap_values[4])

In [ ]:
shap.plots.waterfall(shap_values[7])

**Explaining all instances:**

In [ ]:
shap.summary_plot(shap_values, plot_type="violin")